In [ ]:
import cv2
import numpy as np
import rect
import pytesseract
from PIL import ImageGrab
import time
import matplotlib.pyplot as plt
import img2pdf 
from PIL import Image 
import os 

# add image here.
# We can also use laptop's webcam if the resolution is good enough to capture
# readable document content
path=input("Please enter the path of the image you want to scan :")
image=cv2.imread(path)


# resize image so it can be processed
# choose optimal dimensions such that important content is not lost
image = cv2.resize(image, (800, 775))

# creating copy of original image
orig = image.copy()

# convert to grayscale and blur to smooth
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)


# apply Canny Edge Detection
edged = cv2.Canny(blurred, 70, 200)
orig_edged = edged.copy()

# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
(contours, _) = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

#x,y,w,h = cv2.boundingRect(contours[0])
#cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),0)

# get approximate contour
for c in contours:
    p = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * p, True)

    if len(approx) == 4:
        target = approx
        break


# mapping target points to 800x800 quadrilateral
approx = rect.rectify(target)

pts2 = np.float32([[0,0],[800,0],[800,800],[0,800]])


M = cv2.getPerspectiveTransform(approx,pts2)
dst = cv2.warpPerspective(orig,M,(800,800))


cv2.drawContours(image, [target], -1, (0, 255, 0), 2)
dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)


# using thresholding on warped image to get scanned effect (If Required)

th2 = cv2.adaptiveThreshold(dst,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
th2=cv2.resize(th2, (700, 650))
th3 = cv2.adaptiveThreshold(dst,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
th3=cv2.resize(th3, (700, 650))

cv2.imshow("Original", orig)
cv2.imshow("Original Grayscale", gray)
cv2.imshow("Original Gaussian Blurred", blurred)
cv2.imshow("Canny Edged", orig_edged)
cv2.imshow("Final Contour", image)
cv2.imshow("Adaptive Thresh mean", th2)
cv2.imshow("Adaptive Thresh gauss", th3)
cv2.waitKey(0)
cv2.destroyAllWindows()
filename = 'Original.jpg'
cv2.imwrite(filename, orig)
filename = 'Original Grayscale.jpg'
cv2.imwrite(filename, gray)
filename = 'Original Gaussian Blurred.jpg'
cv2.imwrite(filename, blurred)
filename = 'Canny Edged.jpg'
cv2.imwrite(filename, orig_edged)
filename = 'Final Contour.jpg'
cv2.imwrite(filename, image)
filename = 'Adaptive Thresh mean.jpg'
cv2.imwrite(filename, th2)

titles1 = ["Original" , "Original Grayscale", "Original Gaussian Blurred", "Canny Edged","Final Contour","Adaptive Thresh mean","Adaptive Thresh gauss"]
images1 = [orig,gray,blurred,orig_edged,image,th2,th3]

plt.figure(figsize=(17,5))

plt.subplot(2, 4, 1)
plt.imshow(cv2.cvtColor(images1[0], cv2.COLOR_RGB2BGR))
plt.title(titles1[0])
plt.xticks([])
plt.yticks([])

plt.subplot(2, 4, 2)
plt.imshow(cv2.cvtColor(images1[1], cv2.COLOR_RGB2BGR))
plt.title(titles1[1])
plt.xticks([])
plt.yticks([])

plt.subplot(2, 4, 3)
plt.imshow(cv2.cvtColor(images1[2], cv2.COLOR_RGB2BGR))
plt.title(titles1[2])
plt.xticks([])
plt.yticks([])

plt.subplot(2, 4, 4)
plt.imshow(cv2.cvtColor(images1[3], cv2.COLOR_RGB2BGR))
plt.title(titles1[3])
plt.xticks([])
plt.yticks([])

plt.subplot(2, 4, 5)
plt.imshow(cv2.cvtColor(images1[4], cv2.COLOR_RGB2BGR))
plt.title(titles1[4])
plt.xticks([])
plt.yticks([])

plt.subplot(2, 4, 6)
plt.imshow(cv2.cvtColor(images1[5], cv2.COLOR_RGB2BGR))
plt.title(titles1[5])
plt.xticks([])
plt.yticks([])

plt.subplot(2, 4, 7)
plt.imshow(cv2.cvtColor(images1[6], cv2.COLOR_RGB2BGR))
plt.title(titles1[6])
plt.xticks([])
plt.yticks([])


#OCR:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

img = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
a=pytesseract.image_to_string(img)
print(a)
while True:
    timer = cv2.getTickCount()
     #_,img = img.read()
     #img = captureScreen()
     #DETECTING CHARACTERES
    hImg, wImg,_ = img.shape
    boxes = pytesseract.image_to_data(img)
    for a,b in enumerate(boxes.splitlines()):
        if a!=0:
            b = b.split()
            if len(b)==12:
                x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
                cv2.putText(img,b[11],(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)
                cv2.rectangle(img, (x,y), (x+w, y+h), (50, 50, 255), 2)
    img=cv2.resize(img, (800, 750))
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    break
    
    
    
    
    
    
    
#saving the image as pdf
filename = 'Adaptive Thresh gauss.jpg'
cv2.imwrite(filename, th3)
# storing pdf path
img_path=input("Please enter the path of the image you want to convert as a pdf:")
#C:\Users\hp\Desktop\CV HA\savedImage.jpg


pdf_path=input("Please enter the path where ypu want to save the scanned PDF:")
#C:\Users\hp\Desktop\CV HA\file.pdf
  
# opening image 
image = Image.open(img_path) 
# converting into chunks using img2pdf 
pdf_bytes = img2pdf.convert(image.filename) 
  
# opening or creating pdf file 
file = open(pdf_path, "wb") 
  
# writing pdf files with chunks 
file.write(pdf_bytes) 
  
# closing image file 
image.close() 
  
# closing pdf file 
file.close() 
  
# output 
print("Successfully made pdf file") 






#C:\Users\hp\Desktop\CV HA\testImage.jpg
#C:\Users\hp\Desktop\CV HA\Adaptive Thresh gauss.jpg
#C:\Users\hp\Desktop\CV HA\scanned_document.pdf

Please enter the path of the image you want to scan :C:\Users\hp\Desktop\CV HA\testImage.jpg
a
m a mexican
ezcalere Cocina S100

3714 14th si, suite N
Washington, DC 20010
(202) sos-2114

Dine-In #6
9/25/2017 01:42:20 PM

Placed By Luis
Table #Baro7
Server: Luis

Number of Guests: 1

5.00
9.00
5.00

Corona

E:
1 Hurache Carne Asada
1 Corona

Subtotal: 19.00
Tax 1.90

Total: 20.90
"Online Orders Now Availabe"
visit www. mezcalerodc.com
Immigrants help make America

great!

 


